In [1]:
# !pip install lava-nc
# !pip install networkx==2.8
# !pip install matplotlib
# !pip install cvxpy
# !pip install lava-nc
# !pip install pandas
# !pip install python-louvain

In [2]:
# !pip uninstall networkx -y
# install latest version of networkx
# !pip install networkx==2.8.7

In [1]:
import lava
import cProfile
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from mod_one_exchange_new import mod_one_exchange_approximation_new
from mod_one_exchange import mod_one_exchange_approximation
import graph_col_dimacs_reader as reader
import goemans_williamson_sdp as gw_sdp
import scipy
import matplotlib
import re
import time
import sys
import community as community_louvain

In [2]:
# print version of networkx
print(nx.__version__)

2.8.7


In [3]:
def density(adj_matrix):
    return np.sum(adj_matrix) / (adj_matrix.shape[0] * (adj_matrix.shape[1]-1))

In [4]:
def graph_to_mat(G, path = 'test_graphs/default2.mat'):
    # get the adjancency matrix of the graph
    adj = nx.adjacency_matrix(G).toarray()
    adj = np.asarray(adj, dtype=np.int32)
    scipy.io.savemat(path, {'M': adj})   # M for matrix, to stay consistent with the .mat files we already have
    return

def mat_to_graph(path):
    mat = scipy.io.loadmat(path)
    G = nx.from_numpy_array(mat['M'])
    return G

In [5]:
from GraphColoringTest import gen_permutation_matrix, get_chr, compute_loss, gen_cyclic_graph
import GraphColoringSoln_new as GC
from lava.magma.core.run_configs import Loihi1SimCfg
from lava.magma.core.run_conditions import RunSteps
from lava.proc.monitor.process import Monitor


# define a function graph_painter, input would be a adjacency matrix, output should be a 1-d array of colors
def graph_painter(adj,phi_init):
    # adj is a numpy array
    B = adj
    num_nodes = B.shape[0]
    conns = np.sum(B, axis=1)
    Cscale = 0.1
    C = (1.0 - adj)
    connsC = np.sum(C, axis=1)

    # init_range = 0.95*np.pi
    # phi_init = np.random.uniform(-init_range, init_range, size=(num_nodes,))

    cos_phi_init = np.cos(phi_init)
    sin_phi_init = np.sin(phi_init)

    lr = 0.01

    lrc = lr*(num_nodes - conns)/(num_nodes)
    lrr = Cscale*lr*conns/(num_nodes)
    tau = 500
    decay = 1 - 1/tau
    sigma = 0.01
    num_steps = 2000

    nodes = GC.OScillatoryNeuronBasic(shape=(num_nodes,), phi=phi_init, sigma=sigma, decay=decay, lrc=lrc)
    connections = GC.GraphColorUpdateBasic(shape=(num_nodes,), shape_mat=(num_nodes, num_nodes), B=B)
    phi_monitor = Monitor()

    nodes.cos_out.connect(connections.cos_in)
    nodes.sin_out.connect(connections.sin_in)

    connections.B_cos_out.connect(nodes.B_cos_in)
    connections.B_sin_out.connect(nodes.B_sin_in)
    #connections.C_cos_out.connect(nodes.C_cos_in)
    #connections.C_sin_out.connect(nodes.C_sin_in)
    phi_monitor.probe(nodes.phi, num_steps)
    
    # nodes.run(condition=RunSteps(num_steps=num_steps), run_cfg=Loihi1SimCfg())
    num_run_steps = 3
    num_steps_per_run = 2
    node_wise_color_frequency = np.ones((num_nodes,))
    nodes.color_frequency = node_wise_color_frequency
    for i in range(num_run_steps):
        nodes.run(condition=RunSteps(num_steps=num_steps_per_run), run_cfg=Loihi1SimCfg())
        phi_vals = phi_monitor.get_data()
        phi_history = phi_vals[list(phi_vals.keys())[0]]['phi']
        last_phi = phi_history[-1,:]
        P, order = gen_permutation_matrix(last_phi)
        chromatic_number, min_color_blocks = get_chr(P, adj, order)
        node_wise_color_frequency = np.ones((num_nodes,))
        for node_group in min_color_blocks:
            for node in node_group:
                node_wise_color_frequency[node] = len(node_group)
        
        
    phi_vals = phi_monitor.get_data()
    nodes.stop()
    # print(list(phi_vals.keys()))
    phi_history = phi_vals[list(phi_vals.keys())[0]]['phi']
    # phi_history = phi_vals['Process_0']['phi']

    last_phi = phi_history[-1,:]
    P, order = gen_permutation_matrix(last_phi)
    # chromatic_number, min_color_blocks = get_chr(P, adj, order)
    # print("order-based chromatic number at simulation end: ", chromatic_number)
    # print("color blocks at simulation end: ", min_color_blocks)
    
    print_fr = 500
    chr_ns = []
    min_chr_n = num_nodes
    min_ind = 0
    min_chr_n_color_blocks = [[i] for i in range(num_nodes)]
    for i in range(phi_history.shape[0]):
        # if i%print_fr == print_fr-1:
        #     print("computing for iter: ", i+1)
        #     print("minimum chromatic number: ", min_chr_n)
        curr_phi = phi_history[i,:]
        P, order = gen_permutation_matrix(curr_phi)
        chr_n, chr_n_color_blocks = get_chr(P, adj, order)  # changes here
        if chr_n<min_chr_n:
            min_chr_n = chr_n
            min_ind = i
            min_chr_n_color_blocks = chr_n_color_blocks
        chr_ns.append(chr_n)
    
    # print("minimum order-based chromatic number during simulation: ", min_chr_n)
    # print("color blocks at minimum chromatic number: ", min_chr_n_color_blocks)

    return min_chr_n_color_blocks, phi_history, curr_phi

def get_color_indice_array(color_blocks, adj):
    color_indice_array = np.zeros(adj.shape[0], dtype=np.int32)
    temp_color_index = 0
    for color_block in color_blocks:
        for node in color_block:
            color_indice_array[node] = temp_color_index
        temp_color_index += 1
    num_colors = temp_color_index
    return color_indice_array, num_colors

def get_color_adj_graph(color_indice_array, adj, num_colors):
    color_adj = np.zeros((num_colors,num_colors), dtype=np.int32)
    for i in range(adj.shape[0]):
        for j in range(adj.shape[0]):
            if adj[i][j] == 1:
                color_adj[color_indice_array[i]][color_indice_array[j]] = 1
    return color_adj



In [6]:
def loss_func(phi_history, adj):
    num_nodes = adj.shape[0]
    loss_list = []
    for i in range (len(phi_history)):
        curr_phi = phi_history[i]
        loss = 0
        for j in range (num_nodes):
            for k in range (j):
                if(adj[j][k] == 1):
                    loss -= 1 - np.cos(curr_phi[j] - curr_phi[k])
        loss_list.append(loss)
    return loss_list

def interchange_algo(adj, color_indice_array, num_colors):
    # This shall implement an approximate interchange algorithm from networkx greedy method for graph coloring
    num_nodes = adj.shape[0]
    interchange_list = []
    visited = np.zeros(num_nodes)
    for i in range(num_nodes):
        for j in range(i, num_nodes):
            if(visited[i] == 1 or visited[j] == 1 or i==j):
                continue
            if(color_indice_array[i] != color_indice_array[j]):
                flag = 1
                for k in range(num_nodes):
                    if(k!=j and color_indice_array[k] == color_indice_array[j] and adj[i][k] == 1):
                        flag = 0
                        break
                if flag == 1:
                    for k in range(num_nodes):
                        if(k!=i and color_indice_array[k] == color_indice_array[i] and adj[j][k] == 1):
                            flag = 0
                            break
                if flag == 1:
                    interchange_list.append((i,j))
                    visited[i] = 1
                    visited[j] = 1
                    break       
    return interchange_list

In [7]:
def graph_painter_multi_level(adj, phi_init, level):
    num_nodes = adj.shape[0]
    color_blocks, phi_history, curr_phi = graph_painter(adj, phi_init)
    color_indice_array, num_colors = get_color_indice_array(color_blocks, adj)
    color_adj1 = get_color_adj_graph(color_indice_array, adj, num_colors)
    if(density(color_adj1) == 1):
        print("Level: ", level)
        return color_blocks, phi_history, curr_phi
    else:
        phi_init_adj = 0.7*np.random.uniform(-np.pi, np.pi, size=(num_colors,))
        color_blocks_adj, phi_history_adj, curr_phi_adj = graph_painter_multi_level(color_adj1, phi_init_adj, level+1)
        color_indice_array_adj, num_colors_adj = get_color_indice_array(color_blocks_adj, color_adj1)
        new_color_indice_array = []
        for i in range(len(color_indice_array)):
            new_color_indice_array.append(color_indice_array_adj[color_indice_array[i]])
        new_color_blocks = []
        for i in range(num_colors_adj):
            new_color_blocks.append([])
        for i in range(num_nodes):
            new_color_blocks[new_color_indice_array[i]].append(i)
        return new_color_blocks, phi_history, curr_phi


In [8]:
# huck, myciel3, myciel4, myciel5, myciel6

In [9]:
path = 'test_graphs/myciel4'
# path = 'test_graphs/queen55.mat'
G = mat_to_graph(path)
adj = nx.adjacency_matrix(G).toarray()
num_nodes = adj.shape[0]
init_range = 0.7*np.pi
phi_init = np.random.uniform(-init_range, init_range, size=(num_nodes,))
color_blocks, phi_history, curr_phi = graph_painter_multi_level(adj, phi_init,0)
loss_list = loss_func(phi_history, adj)
print("Initial Loss: ", loss_list[0])
print("Final Loss: ", loss_list[-1])
color_indice_array, num_colors = get_color_indice_array(color_blocks, adj)
color_adj1 = get_color_adj_graph(color_indice_array, adj, num_colors)
print("num_colors: ", num_colors)
print("Density of the original graph: ", density(adj))
print("Density of the colored graph: ", density(color_adj1))
# Plotting the loss
plt.plot(loss_list)
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.title('Loss vs Iterations')
plt.show()

interchange_list = interchange_algo(adj, color_indice_array,num_colors)
print("Number of interchanges: ", len(interchange_list))
for swap_pairs in interchange_list:
    curr_phi[swap_pairs[0]], curr_phi[swap_pairs[1]] = curr_phi[swap_pairs[1]], curr_phi[swap_pairs[0]]
color_blocks, phi_history, curr_phi = graph_painter_multi_level(adj,curr_phi,0)
loss_list = loss_func(phi_history, adj)
print("Loss: ", loss_list[-1])
color_indice_array, num_colors = get_color_indice_array(color_blocks, adj)
color_adj1 = get_color_adj_graph(color_indice_array,adj, num_colors)
print("num_colors: ", num_colors)
print("Density of the original graph: ", density(adj))
print("Density of the colored graph: ", density(color_adj1))
# Plotting the loss
plt.plot(loss_list)
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.title('Loss vs Iterations')
plt.show()


/tmp/ipykernel_8705/133872283.py:4: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj = nx.adjacency_matrix(G).toarray()


AttributeError: 'numpy.ndarray' object has no attribute 'external_pipe_flag'